In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16


In [2]:
SIZE = 200  #Resize images

# training data 
train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/drive/MyDrive/cattle_dis_augmnt/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

/content/drive/MyDrive/cattle_dis_augmnt/ringworm
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download.png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (1).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (2).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (3).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (4).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (5).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (6).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (7).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (8).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (9).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (10).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (11).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (12).png
/content/drive/MyDrive/cattle_dis_augmnt/ringworm/download (13).png
/content/dri

In [3]:
#Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [4]:
train_labels[0]

'/content/drive/MyDrive/cattle_dis_augmnt/ringworm'

In [5]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#le.fit(test_labels)
#test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels_encoded, test_size=0.20, random_state=42)

In [7]:
SIZE = 200 
VGG_model_16 = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))


58889256/58889256 [==============================] - 3s 0us/step


In [8]:
VGG_model_16.trainable = False

model1 = Sequential()
model1.add(VGG_model_16)
model1.add(Flatten())
model1.add(Dense(256,input_dim=3,activation='relu'))
model1.add(Dense(128,activation='relu'))
model1.add(Dense(64,activation='relu'))
model1.add(Dense(3,activation='softmax'))

In [9]:
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [10]:
model1.fit(X_train,y_train,validation_split=0.20, epochs=20)
#Require hyperparameter tuning for improvement

Epoch 1/20
11/11 [==============================] - 17s 464ms/step - loss: 12.0645 - accuracy: 0.4554 - val_loss: 4.6239 - val_accuracy: 0.5488
Epoch 2/20
11/11 [==============================] - 1s 125ms/step - loss: 1.7517 - accuracy: 0.8123 - val_loss: 2.8209 - val_accuracy: 0.6829
Epoch 3/20
11/11 [==============================] - 1s 124ms/step - loss: 0.2078 - accuracy: 0.9600 - val_loss: 3.8143 - val_accuracy: 0.6829
Epoch 4/20
11/11 [==============================] - 1s 128ms/step - loss: 0.0570 - accuracy: 0.9785 - val_loss: 3.5497 - val_accuracy: 0.7561
Epoch 5/20
11/11 [==============================] - 1s 127ms/step - loss: 0.0166 - accuracy: 0.9877 - val_loss: 3.4172 - val_accuracy: 0.7561
Epoch 6/20
11/11 [==============================] - 1s 125ms/step - loss: 0.0275 - accuracy: 0.9908 - val_loss: 3.1284 - val_accuracy: 0.7805
Epoch 7/20
11/11 [==============================] - 1s 124ms/step - loss: 0.0126 - accuracy: 0.9969 - val_loss: 3.1890 - val_accuracy: 0.7683
Epoc

Now if you want to freez some of the conv layers of VGG then try this

In [12]:
base_model = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(200,200,3)
)

In [13]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [14]:
base_model.trainable = True

set_trainable = False

for layer in base_model.layers:
  if layer.name == 'block4_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

for layer in base_model.layers:
  print(layer.name,layer.trainable)

input_2 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [15]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [17]:
model2 = Sequential()
model2.add(base_model)
model2.add(Flatten())
model2.add(Dense(256,input_dim=3,activation='relu'))
model2.add(Dense(128,activation='relu'))
model2.add(Dense(64,activation='relu'))
model2.add(Dense(3,activation='softmax'))

In [ ]:
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model1.fit(X_train,y_train,validation_split=0.20, epochs=20)
